# Introduction: Automated Feature Engineering Basics
In this notebook, we will walk through applying automated feature engineering to the Home Credit Default Risk dataset using the featuretools library. Featuretools is an open-source Python package for automatically creating new features from multiple tables of structured, related data. It is ideal tool for problems such as the Home Credit Default Risk competition where there are several related tables that need to be combined into a single dataframe for training (and one for testing).

# Feature Engineering
The objective of feature engineering is to create new features (alos called explantory variables or predictors) to represent as much information from an entire dataset in one table. Typically, this process is done by hand using pandas operations such as groupby, agg, or merge and can be very tedious. Moreover, manual feature engineering is limited both by human time constraints and imagination: we simply cannot conceive of every possible feature that will be useful. (For an example of using manual feature engineering, check out part one and part two applied to this competition). The importance of creating the proper features cannot be overstated because a machine learning model can only learn from the data we give to it. Extracting as much information as possible from the available datasets is crucial to creating an effective solution.

Automated feature engineering aims to help the data scientist with the problem of feature creation by automatically building hundreds or thousands of new features from a dataset. Featuretools - the only library for automated feature engineering at the moment - will not replace the data scientist, but it will allow her to focus on more valuable parts of the machine learning pipeline, such as delivering robust models into production.

Here we will touch on the concepts of automated feature engineering with featuretools and show how to implement it for the Home Credit Default Risk competition. We will stick to the basics so we can get the ideas down and then build upon this foundation in later work when we customize featuretools. We will work with a subset of the data because this is a computationally intensive job that is outside the capabilities of the Kaggle kernels. I took the work done in this notebook and ran the methods on the entire dataset with the results available here. At the end of this notebook, we'll look at the features themselves, as well as the results of modeling with different combinations of hand designed and automatically built features.

If you are new to this competition, I suggest checking out this post to get started. For a good take on why features are so important, here's a blog post by one of the developers of Featuretools.

# Libraries

In [1]:
# pandas and numpy for data manipulation
import pandas as pd
import numpy as np

# featuretools for automated feature engineering
import featuretools as ft

# matplotlit and seaborn for visualizations
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 22
import seaborn as sns

# Suppress warnings from pandas
import warnings
warnings.filterwarnings('ignore')

# 1.0.0 Read in Data and Create Small Datasets

We will read in the full dataset, sort by the SK_ID_CURR and keep only the first 1000 rows to make the calculations feasible. Later we can convert to a script and run with the entire datasets.

In [4]:
# Read in the datasets and limit to the first 1000 rows (sorted by SK_ID_CURR) 
# This allows us to actually see the results in a reasonable amount of time! 
app_train = pd.read_csv('application_train.csv').sort_values('SK_ID_CURR').reset_index(drop = True).loc[:1000, :]
app_test = pd.read_csv('application_test.csv').sort_values('SK_ID_CURR').reset_index(drop = True).loc[:1000, :]
bureau = pd.read_csv('bureau.csv').sort_values(['SK_ID_CURR', 'SK_ID_BUREAU']).reset_index(drop = True).loc[:1000, :]
bureau_balance = pd.read_csv('bureau_balance.csv').sort_values('SK_ID_BUREAU').reset_index(drop = True).loc[:1000, :]
cash = pd.read_csv('POS_CASH_balance.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop = True).loc[:1000, :]
credit = pd.read_csv('credit_card_balance.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop = True).loc[:1000, :]
previous = pd.read_csv('previous_application.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop = True).loc[:1000, :]
installments = pd.read_csv('installments_payments.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop = True).loc[:1000, :]

We'll join the train and test set together but add a separate column identifying the set. This is important because we are going to want to apply the same exact procedures to each dataset. It's safest to just join them together and treat them as a single dataframe.

(I'm not sure if this is allowing data leakage into the train set and if these feature creation operations should be applied separately. Any thoughts would be much appreciated!)

In [5]:
# Add identifying column
app_train['set'] = 'train'
app_test['set'] = 'test'
app_test["TARGET"] = np.nan

# Append the dataframes
app = app_train.append(app_test, ignore_index = True)

# 1.0.1 Featuretools Basics

Featuretools Basics
Featuretools is an open-source Python library for automatically creating features out of a set of related tables using a technique called deep feature synthesis. Automated feature engineering, like many topics in machine learning, is a complex subject built upon a foundation of simpler ideas. By going through these ideas one at a time, we can build up our understanding of how featuretools which will later allow for us to get the most out of it.

There are a few concepts that we will cover along the way:

Entities and EntitySets

Relationships between tables

Feature primitives: aggregations and transformations

Deep feature synthesis

# 1.0.2 Entities and Entitysets
An entity is simply a table or in Pandas, a dataframe. The observations are in the rows and the features in the columns. An entity in featuretools must have a unique index where none of the elements are duplicated. Currently, only app, bureau, and previous have unique indices (SK_ID_CURR, SK_ID_BUREAU, and SK_ID_PREV respectively). For the other dataframes, we must pass in make_index = True and then specify the name of the index. Entities can also have time indices where each entry is identified by a unique time. (There are not datetimes in any of the data, but there are relative times, given in months or days, that we could consider treating as time variables).

An EntitySet is a collection of tables and the relationships between them. This can be thought of a data structute with its own methods and attributes. Using an EntitySet allows us to group together multiple tables and manipulate them much quicker than individual tables.

First we'll make an empty entityset named clients to keep track of all the data.

In [6]:
# Entity set with id applications
es = ft.EntitySet(id = 'clients')

Now we define each entity, or table of data. We need to pass in an index if the data has one or make_index = True if not. Featuretools will automatically infer the types of variables, but we can also change them if needed. For intstance, if we have a categorical variable that is represented as an integer we might want to let featuretools know the right type.

In [8]:
# Entities with a unique index
es = es.entity_from_dataframe(entity_id = 'app', dataframe = app, index = 'SK_ID_CURR')

es = es.entity_from_dataframe(entity_id = 'bureau', dataframe = bureau, index = 'SK_ID_BUREAU')

es = es.entity_from_dataframe(entity_id = 'previous', dataframe = previous, index = 'SK_ID_PREV')

# Entities that do not have a unique index
es = es.entity_from_dataframe(entity_id = 'bureau_balance', dataframe = bureau_balance, 
                              make_index = True, index = 'bureaubalance_index')

es = es.entity_from_dataframe(entity_id = 'cash', dataframe = cash, 
                              make_index = True, index = 'cash_index')

es = es.entity_from_dataframe(entity_id = 'installments', dataframe = installments,
                              make_index = True, index = 'installments_index')

es = es.entity_from_dataframe(entity_id = 'credit', dataframe = credit,
                              make_index = True, index = 'credit_index')

In [9]:
es

Entityset: clients
  Entities:
    app [Rows: 2002, Columns: 123]
    bureau [Rows: 1001, Columns: 17]
    previous [Rows: 1001, Columns: 37]
    bureau_balance [Rows: 1001, Columns: 4]
    cash [Rows: 1001, Columns: 9]
    installments [Rows: 1001, Columns: 9]
    credit [Rows: 1001, Columns: 24]
  Relationships:
    No relationships

# 1.0.3 Relationships
Relationships are a fundamental concept not only in featuretools, but in any relational database. The best way to think of a one-to-many relationship is with the analogy of parent-to-child. A parent is a single individual, but can have mutliple children. The children can then have multiple children of their own. In a parent table, each individual has a single row. Each individual in the parent table can have multiple rows in the child table.

As an example, the app dataframe has one row for each client (SK_ID_CURR) while the bureau dataframe has multiple previous loans (SK_ID_PREV) for each parent (SK_ID_CURR). Therefore, the bureau dataframe is the child of the app dataframe. The bureau dataframe in turn is the parent of bureau_balance because each loan has one row in bureau but multiple monthly records in bureau_balance.

In [10]:
print('Parent: app, Parent Variable: SK_ID_CURR\n\n', app.iloc[:, 111:115].head())
print('\nChild: bureau, Child Variable: SK_ID_CURR\n\n', bureau.iloc[10:30, :4].head())

Parent: app, Parent Variable: SK_ID_CURR

    SK_ID_CURR  TARGET  TOTALAREA_MODE WALLSMATERIAL_MODE
0      100002     1.0          0.0149       Stone, brick
1      100003     0.0          0.0714              Block
2      100004     0.0             NaN                NaN
3      100006     0.0             NaN                NaN
4      100007     0.0             NaN                NaN

Child: bureau, Child Variable: SK_ID_CURR

     SK_ID_CURR  SK_ID_BUREAU CREDIT_ACTIVE CREDIT_CURRENCY
10      100002       6158905        Closed      currency 1
11      100002       6158906        Closed      currency 1
12      100002       6158907        Closed      currency 1
13      100002       6158908        Closed      currency 1
14      100002       6158909        Active      currency 1


The SK_ID_CURR "100002" has one row in the parent table and multiple rows in the child.

Two tables are linked via a shared variable. The app and bureau dataframe are linked by the SK_ID_CURR variable while the bureau and bureau_balance dataframes are linked with the SK_ID_BUREAU. Defining the relationships is relatively straightforward, and the diagram provided by the competition is helpful for seeing the relationships. For each relationship, we need to specify the parent variable and the child variable. Altogether, there are a total of 6 relationships between the tables. Below we specify all six relationships and then add them to the EntitySet.

In [11]:
# Relationship between app and bureau
r_app_bureau = ft.Relationship(es['app']['SK_ID_CURR'], es['bureau']['SK_ID_CURR'])

# Relationship between bureau and bureau balance
r_bureau_balance = ft.Relationship(es['bureau']['SK_ID_BUREAU'], es['bureau_balance']['SK_ID_BUREAU'])

# Relationship between current app and previous apps
r_app_previous = ft.Relationship(es['app']['SK_ID_CURR'], es['previous']['SK_ID_CURR'])

# Relationships between previous apps and cash, installments, and credit
r_previous_cash = ft.Relationship(es['previous']['SK_ID_PREV'], es['cash']['SK_ID_PREV'])
r_previous_installments = ft.Relationship(es['previous']['SK_ID_PREV'], es['installments']['SK_ID_PREV'])
r_previous_credit = ft.Relationship(es['previous']['SK_ID_PREV'], es['credit']['SK_ID_PREV'])

In [12]:
# Add in the defined relationships
es = es.add_relationships([r_app_bureau, r_bureau_balance, r_app_previous,
                           r_previous_cash, r_previous_installments, r_previous_credit])
# Print out the EntitySet
es

Entityset: clients
  Entities:
    app [Rows: 2002, Columns: 123]
    bureau [Rows: 1001, Columns: 17]
    previous [Rows: 1001, Columns: 37]
    bureau_balance [Rows: 1001, Columns: 4]
    cash [Rows: 1001, Columns: 9]
    installments [Rows: 1001, Columns: 9]
    credit [Rows: 1001, Columns: 24]
  Relationships:
    bureau.SK_ID_CURR -> app.SK_ID_CURR
    bureau_balance.SK_ID_BUREAU -> bureau.SK_ID_BUREAU
    previous.SK_ID_CURR -> app.SK_ID_CURR
    cash.SK_ID_PREV -> previous.SK_ID_PREV
    installments.SK_ID_PREV -> previous.SK_ID_PREV
    credit.SK_ID_PREV -> previous.SK_ID_PREV

Slightly advanced note: we need to be careful to not create a diamond graph where there are multiple paths from a parent to a child. If we directly link app and cash via SK_ID_CURR; previous and cash via SK_ID_PREV; and app and previous via SK_ID_CURR, then we have created two paths from app to cash. This results in ambiguity, so the approach we have to take instead is to link app to cash through previous. We establish a relationship between previous (the parent) and cash (the child) using SK_ID_PREV. Then we establish a relationship between app (the parent) and previous (now the child) using SK_ID_CURR. Then featuretools will be able to create features on app derived from both previous and cash by stacking multiple primitives.

All entities in the entity can be related to each other. In theory this allows us to calculate features for any of the entities, but in practice, we will only calculate features for the app dataframe since that will be used for training/testing.

# 1.0.4 Feature Primitives
A feature primitive is an operation applied to a table or a set of tables to create a feature. These represent simple calculations, many of which we already use in manual feature engineering, that can be stacked on top of each other to create complex features. Feature primitives fall into two categories:

Aggregation: function that groups together child datapoints for each parent and then calculates a statistic such as mean, min, max, or standard deviation. An example is calculating the maximum previous loan amount for each client. An aggregation works across multiple tables using relationships between tables.
Transformation: an operation applied to one or more columns in a single table. An example would be taking the absolute value of a column, or finding the difference between two columns in one table.
A list of the available features primitives in featuretools can be viewed below.

In [13]:
# List the primitives in a dataframe
primitives = ft.list_primitives()
pd.options.display.max_colwidth = 100
primitives[primitives['type'] == 'aggregation'].head(10)

name         type  \
0   percent_true  aggregation   
1          trend  aggregation   
2            any  aggregation   
3     num_unique  aggregation   
4            all  aggregation   
5  n_most_common  aggregation   
6           mode  aggregation   
7       num_true  aggregation   
8            std  aggregation   
9            max  aggregation   

                                                               description  
0                 Finds the percent of 'True' values in a boolean feature.  
1           Calculates the slope of the linear trend of variable overtime.  
2                                             Test if any value is 'True'.  
3                      Returns the number of unique categorical variables.  
4                                           Test if all values are 'True'.  
5               Finds the N most common elements in a categorical feature.  
6                  Finds the most common element in a categorical feature.  
7                          Finds the number of 'True' values in a boolean.  
8  Finds the standard deviation of a numeric feature ignoring null values.  
9                   Finds the maximum non-null value of a numeric feature.

In [14]:
primitives[primitives['type'] == 'transform'].head(10)

name       type  \
19    numwords  transform   
20  characters  transform   
21     seconds  transform   
22    subtract  transform   
23       month  transform   
24         day  transform   
25    latitude  transform   
26      divide  transform   
27        isin  transform   
28     minutes  transform   

                                                                     description  
19                   Returns the words in a given string by counting the spaces.  
20                                      Return the characters in a given string.  
21                     Transform a Timedelta feature into the number of seconds.  
22                      Creates a transform feature that subtracts two features.  
23                                  Transform a Datetime feature into the month.  
24                                    Transform a Datetime feature into the day.  
25                            Returns the first value of the tuple base feature.  
26                        Creates a transform feature that divides two features.  
27  For each value of the base feature, checks whether it is in a provided list.  
28                     Transform a Timedelta feature into the number of minutes.

# 1.0.5 Deep Feature Synthesis

Deep Feature Synthesis (DFS) is the process featuretools uses to make new features. DFS stacks feature primitives to form features with a "depth" equal to the number of primitives. For example, if we take the maximum value of a client's previous loans (say MAX(previous.loan_amount)), that is a "deep feature" with a depth of 1. To create a feature with a depth of two, we could stack primitives by taking the maximum value of a client's average montly payments per previous loan (such as MAX(previous(MEAN(installments.payment)))). The original paper on automated feature engineering using deep feature synthesis is worth a read.

To perform DFS in featuretools, we use the dfs function passing it an entityset, the target_entity (where we want to make the features), the agg_primitives to use, the trans_primitives to use and the max_depth of the features. Here we will use the default aggregation and transformation primitives, a max depth of 2, and calculate primitives for the app entity. Because this process is computationally expensive, we can run the function using features_only = True to return only a list of the features and not calculate the features themselves. This can be useful to look at the resulting features before starting an extended computation.

# 1.0.6 DFS with Default Primitives

In [15]:
# Default primitives from featuretools
default_agg_primitives =  ["sum", "std", "max", "skew", "min", "mean", "count", "percent_true", "num_unique", "mode"]
default_trans_primitives =  ["day", "year", "month", "weekday", "haversine", "numwords", "characters"]

# DFS with specified primitives
feature_names = ft.dfs(entityset = es, target_entity = 'app',
                       trans_primitives = default_trans_primitives,
                       agg_primitives=default_agg_primitives, 
                       max_depth = 2, features_only=True)

print('%d Total Features' % len(feature_names))

1697 Total Features


In [18]:
#DFS with default primitives
feature_matrix, feature_names = ft.dfs(entityset = es, target_entity = 'app',
                                       trans_primitives = default_trans_primitives,
                                       agg_primitives=default_agg_primitives, 
                                        max_depth = 2, features_only=False, verbose = True)

pd.options.display.max_columns = 1700
feature_matrix.head(10)

Built 1697 features
Elapsed: 01:02 | Remaining: 00:00 | Progress: 100%|██████████████████████████████████████████| Calculated: 11/11 chunks


AMT_ANNUITY  AMT_CREDIT  AMT_GOODS_PRICE  AMT_INCOME_TOTAL  \
SK_ID_CURR                                                               
100001          20560.5    568800.0         450000.0          135000.0   
100002          24700.5    406597.5         351000.0          202500.0   
100003          35698.5   1293502.5        1129500.0          270000.0   
100004           6750.0    135000.0         135000.0           67500.0   
100005          17370.0    222768.0         180000.0           99000.0   
100006          29686.5    312682.5         297000.0          135000.0   
100007          21865.5    513000.0         513000.0          121500.0   
100008          27517.5    490495.5         454500.0           99000.0   
100009          41301.0   1560726.0        1395000.0          171000.0   
100010          42075.0   1530000.0        1530000.0          360000.0   

            AMT_REQ_CREDIT_BUREAU_DAY  AMT_REQ_CREDIT_BUREAU_HOUR  \
SK_ID_CURR                                                          
100001                            0.0                         0.0   
100002                            0.0                         0.0   
100003                            0.0                         0.0   
100004                            0.0                         0.0   
100005                            0.0                         0.0   
100006                            NaN                         NaN   
100007                            0.0                         0.0   
100008                            0.0                         0.0   
100009                            0.0                         0.0   
100010                            0.0                         0.0   

            AMT_REQ_CREDIT_BUREAU_MON  AMT_REQ_CREDIT_BUREAU_QRT  \
SK_ID_CURR                                                         
100001                            0.0                        0.0   
100002                            0.0                        0.0   
100003                            0.0                        0.0   
100004                            0.0                        0.0   
100005                            0.0                        0.0   
100006                            NaN                        NaN   
100007                            0.0                        0.0   
100008                            0.0                        1.0   
100009                            1.0                        1.0   
100010                            0.0                        0.0   

            AMT_REQ_CREDIT_BUREAU_WEEK  AMT_REQ_CREDIT_BUREAU_YEAR  \
SK_ID_CURR                                                           
100001                             0.0                         0.0   
100002                             0.0                         1.0   
100003                             0.0                         0.0   
100004                             0.0                         0.0   
100005                             0.0                         3.0   
100006                             NaN                         NaN   
100007                             0.0                         0.0   
100008                             0.0                         1.0   
100009                             0.0                         2.0   
100010                             0.0                         0.0   

            APARTMENTS_AVG  APARTMENTS_MEDI  APARTMENTS_MODE  \
SK_ID_CURR                                                     
100001              0.0660           0.0666           0.0672   
100002              0.0247           0.0250           0.0252   
100003              0.0959           0.0968           0.0924   
100004                 NaN              NaN              NaN   
100005                 NaN              NaN              NaN   
100006                 NaN              NaN              NaN   
100007                 NaN              NaN              NaN   
100008                 NaN              NaN              NaN

In [19]:
feature_names[-20:]

[<Feature: SKEW(bureau.MEAN(bureau_balance.MONTHS_BALANCE))>,
 <Feature: SKEW(bureau.COUNT(bureau_balance))>,
 <Feature: SKEW(bureau.NUM_UNIQUE(bureau_balance.STATUS))>,
 <Feature: MIN(bureau.SUM(bureau_balance.MONTHS_BALANCE))>,
 <Feature: MIN(bureau.STD(bureau_balance.MONTHS_BALANCE))>,
 <Feature: MIN(bureau.MAX(bureau_balance.MONTHS_BALANCE))>,
 <Feature: MIN(bureau.SKEW(bureau_balance.MONTHS_BALANCE))>,
 <Feature: MIN(bureau.MEAN(bureau_balance.MONTHS_BALANCE))>,
 <Feature: MIN(bureau.COUNT(bureau_balance))>,
 <Feature: MIN(bureau.NUM_UNIQUE(bureau_balance.STATUS))>,
 <Feature: MEAN(bureau.SUM(bureau_balance.MONTHS_BALANCE))>,
 <Feature: MEAN(bureau.STD(bureau_balance.MONTHS_BALANCE))>,
 <Feature: MEAN(bureau.MAX(bureau_balance.MONTHS_BALANCE))>,
 <Feature: MEAN(bureau.SKEW(bureau_balance.MONTHS_BALANCE))>,
 <Feature: MEAN(bureau.MIN(bureau_balance.MONTHS_BALANCE))>,
 <Feature: MEAN(bureau.MEAN(bureau_balance.MONTHS_BALANCE))>,
 <Feature: MEAN(bureau.COUNT(bureau_balance))>,
 <Feat

DFS with Selected Aggregation Primitives
With featuretools, we were able to go from 121 original features to almost 1700 in a few lines of code. When I did feature engineering by hand, it took about 12 hours to create a comparable size dataset. However, while we get a lot of features in featuretools, this function call is not very well-informed. We simply used the default aggregations without thinking about which ones are "important" for the problem. We end up with a lot of features, but they are probably not all relevant to the problem. Too many irrelevant features can decrease performance by drowning out the important features (related to the curse of dimensionality)

The next call we make will specify a smaller set of features. We still are not using much domain knowledge, but this feature set will be more manageable. The next step from here is improving the features we actually build and performing feature selection.

In [20]:
# Specify the aggregation primitives
feature_matrix_spec, feature_names_spec = ft.dfs(entityset = es, target_entity = 'app',  
                                                 agg_primitives = ['sum', 'count', 'min', 'max', 'mean', 'mode'], 
                                                 max_depth = 2, features_only = False, verbose = True)

Built 884 features
Elapsed: 00:24 | Remaining: 00:00 | Progress: 100%|██████████████████████████████████████████| Calculated: 11/11 chunks


That "only" gives us 884 features (and takes about 12 hours to run on the complete dataset).

In [21]:
pd.options.display.max_columns = 1000
feature_matrix_spec.head(10)

AMT_ANNUITY  AMT_CREDIT  AMT_GOODS_PRICE  AMT_INCOME_TOTAL  \
SK_ID_CURR                                                               
100001          20560.5    568800.0         450000.0          135000.0   
100002          24700.5    406597.5         351000.0          202500.0   
100003          35698.5   1293502.5        1129500.0          270000.0   
100004           6750.0    135000.0         135000.0           67500.0   
100005          17370.0    222768.0         180000.0           99000.0   
100006          29686.5    312682.5         297000.0          135000.0   
100007          21865.5    513000.0         513000.0          121500.0   
100008          27517.5    490495.5         454500.0           99000.0   
100009          41301.0   1560726.0        1395000.0          171000.0   
100010          42075.0   1530000.0        1530000.0          360000.0   

            AMT_REQ_CREDIT_BUREAU_DAY  AMT_REQ_CREDIT_BUREAU_HOUR  \
SK_ID_CURR                                                          
100001                            0.0                         0.0   
100002                            0.0                         0.0   
100003                            0.0                         0.0   
100004                            0.0                         0.0   
100005                            0.0                         0.0   
100006                            NaN                         NaN   
100007                            0.0                         0.0   
100008                            0.0                         0.0   
100009                            0.0                         0.0   
100010                            0.0                         0.0   

            AMT_REQ_CREDIT_BUREAU_MON  AMT_REQ_CREDIT_BUREAU_QRT  \
SK_ID_CURR                                                         
100001                            0.0                        0.0   
100002                            0.0                        0.0   
100003                            0.0                        0.0   
100004                            0.0                        0.0   
100005                            0.0                        0.0   
100006                            NaN                        NaN   
100007                            0.0                        0.0   
100008                            0.0                        1.0   
100009                            1.0                        1.0   
100010                            0.0                        0.0   

            AMT_REQ_CREDIT_BUREAU_WEEK  AMT_REQ_CREDIT_BUREAU_YEAR  \
SK_ID_CURR                                                           
100001                             0.0                         0.0   
100002                             0.0                         1.0   
100003                             0.0                         0.0   
100004                             0.0                         0.0   
100005                             0.0                         3.0   
100006                             NaN                         NaN   
100007                             0.0                         0.0   
100008                             0.0                         1.0   
100009                             0.0                         2.0   
100010                             0.0                         0.0   

            APARTMENTS_AVG  APARTMENTS_MEDI  APARTMENTS_MODE  \
SK_ID_CURR                                                     
100001              0.0660           0.0666           0.0672   
100002              0.0247           0.0250           0.0252   
100003              0.0959           0.0968           0.0924   
100004                 NaN              NaN              NaN   
100005                 NaN              NaN              NaN   
100006                 NaN              NaN              NaN   
100007                 NaN              NaN              NaN   
100008                 NaN              NaN              NaN

# 1.0.7 Notes on Basic Implementation
These calls represent only a small fraction of the ability of featuretools. We did not specify the variable types when creating entities, did not use the relative time variables, and didn't touch on custom primitives or seed features or interesting values! Nonetheless, in this notebook, we were able to learn the basic foundations which will allow us to more effective use the tool as we learn how it works. Now, let's take a look at some of the features we have built and modeling results.

# 2.0.0 Results
To determine whether our basic implementation of featuretools was useful, we can look at several results:

Cross validation scores and public leaderboard scores using several different sets of features.
Correlations: both between the features and the TARGET, and between features themselves
Feature importances: determined by a gradient boosting machine model

# 3.0.0 Correlations
Next we can look at correlations within the data. When we look at correlations with the target, we need to be careful about the multiple comparisons problem: if we make a ton of features, some are likely to be correlated with the target simply because of random noise. Using correlations is fine as a first approximation for identifying "good features", but it is not a rigorous feature selection method.

Also, based on examining some of the features, it seems there might be issues with collinearity between features made by featuretools. Features that are highly correlated with one another can diminish interpretability and generalization performance on the test set. In an ideal scenario, we would have a set of independent features, but that rarely occurs in practice. If there are very highly correlated varibables, we might want to think about removing some of them.

For the correlations, we will focus on the feature_matrix_spec, the features we made by specifying the primitives. The same analysis could be applied to the default feature set. These correlations were calculated using the entire training section of the feature matrix.

# End